# 1 Leiðarval hjá Strætó
Í þessu verkefni á að búa til leitarvél fyrir Strætó, notast verður við raungögn frá Strætó
sem er að finna á
https://straeto.is/um-straeto/opin-rafraen-gogn.
Til einföldunar munuð þið nota skrána gtfs.zip sem er að finna á Canvas síðu námskeiðsins til að tryggja að
allir noti sömu útgáfu.


## 2.1 Inntak
Inntakið er tímatafla strætó, listi yfir leiðir, vagna og stoppistöðvar. Við skilgreinum netið
G á eftirfarandi hátt. Hver einasta stoppistöð verður hnútur.
Ef vagn ekur á milli stoppistöðva x og y á tímum t1 og t2 með engum öðrum stoppum á
milli þá búum við til nýjan hnút v með leggina (x, v), (v, y) og merkjum þá með tímunum t_1 og t_2. Hnúturinn v geymir upplýsingar um hvaða vagni sá hnútur tilheyrir, athugið þessi
hnútur verður ekki endurnýttur.
Fyrir leiðina sem er sýnd á myndinni þá samsvarar hún fimm hnútum. Hnútar 1, 3, 5
eru stoppistöðvarnar Hlemmur B, Sólfarið og Harpa. Hnútur 2 tengir saman Hlemm B og
Sólfarið og leggirnir hafa tímann 10:22 og 10:25. Hnútur 4 tengir saman Sólfarið og Hörpu,
fyrri leggurinn hefur tímann 10:25 (ekki sýnilegt á mynd) og sá seinni 10:26. Þetta er dæmi
um löglegan veg þar sem allir leggir samsvara vögnum sem við getum tekið og tímarnir
stemma, þ.e. eru í vaxandi röð

## 2.2 Reiknirit
Einfaldasta tilfellið sem við viljum leysa þá fá um við par af stoppistöðvum og tíma, (x, y, t)
og úttakið er leið í strætókerfinu frá x til y sem fer frá x á tíma t′ ≥ t og lágmarkar
komutímann á áfangastað. Ef við skiptum um strætisvagn þá samsvarar það því að "bíða"á
hnútnum sem er á stoppistöð og taka legg sem tilheyrir öðrum vagni heldur en þeim sem
við komum með.
Til að leysa þetta verkefni er hægt að nota reiknirit Dijkstra með einni breytingu, í stað
þess að slaka (RELAX aðferðin) á leggjum með því að leggja saman vigtir þá notum við
komutímann á leggnum ef leggurinn er löglegur, þ.e. ef (u, v) er leggurinn sem við skoðum
með tímann t þá setjum við d[v] = min(d[v], t) ef t ≥ d[u].


# 2.3 Verkþættir
## 2.3.1 Þáttun (⋆)
Gögnin fyrir strætó eru gefin á GTFS formi, lýsingu á GTFS er að finna á https://gtfs.
org/schedule/reference/. Til að búa til netið þarf að þátta (e. parse) gögnin, þ.e. lesa
þau inn og tengja saman á skynsamlegan hátt.

Til einföldunar ætlum við eingöngu að nota eftirfarandi hluta af gögnunum

agency_id=1 eingöngu leiðir sem tilheyra Strætó BS merktar ST.*

Leiðirnar eru ólíkar milli vikudaga, gefum okkur að við séum að vinna með virka daga
og sleppum næturstrætó (leiðir 101-106). Það má líka sleppa helgidögum, þ.e. sem er
skilgreint í calendar_dates.csv skráni.

Sumar leiðir eru í pöntunarþjónustu, við höldum eingöngu þeim leggjum sem eru með
pickup_type=0 í stop_times.csv.

#### import

In [22]:
import pandas as pd
from datetime import datetime, timedelta
import heapq

#### Hjálparföll

In [23]:
# fall til að finna út hvort við séum með tvö id's með sama nafn
def find_unique_stop_ids(df, stop_name):
    filtered_df = df[df['stop_name'] == stop_name]
    unique_stop_ids = filtered_df['stop_id'].unique()
    return unique_stop_ids

In [24]:
# Lesa inn gögnin
stops_df = pd.read_csv('content/stops.txt')
trips_df = pd.read_csv('content/trips.txt')
stop_times_df = pd.read_csv('content/stop_times.txt')
routes_df = pd.read_csv('content//routes.txt', comment='#')
calendar_dates_df = pd.read_csv('content/calendar_dates.txt')
# næturstrætó
night_buses = ['101', '102', '103', '104', '105', '106']

# Filter -> bara sem byrjar á ST. & agency_id = 1 & er ekki næturstrætó
routes_df = routes_df[(routes_df['route_id'].str.startswith("ST.", na=False)) &
                      (routes_df['agency_id'] == 1) &
                      (~routes_df['route_short_name'].isin(night_buses))]

# Bæta við dálki sem hefur aðeins upplýsingar um vikudaga.
trips_df['day_pattern'] = trips_df['service_id'].apply(lambda x: x.split('_')[1])

# Filter -> bara virkir dagar.
def all_weekdays_no_weekends(pattern):
    weekdays = 'MTWTF'
    return all(day in pattern for day in weekdays if day != '-')
    
filtered_trips_df = trips_df[trips_df['day_pattern'].apply(all_weekdays_no_weekends)]

# Filter -> pickup_type = 0
filtered_stop_times_df = stop_times_df[stop_times_df['pickup_type'] == 0]

# Merge -> filtered_stop_times & stops til að fá nafn á stoppi
stop_times_with_stops_df = pd.merge(filtered_stop_times_df, stops_df, on='stop_id')

# Merge -> stop_times_with_stops & trips til að fá upplýsingar um ferðir
stop_times_with_stops_and_trips_df = pd.merge(stop_times_with_stops_df, filtered_trips_df, on='trip_id')

# Merge -> Henda þessu í öllu í eina klessu
stop_times_with_stops_trips_and_routes_df = pd.merge(stop_times_with_stops_and_trips_df, routes_df[['route_id', 'route_short_name']], on='route_id')

# Merge -> stop_times_with_stops_trips_and_routes & calendar_dates til að fá upplýsingar um dagsetningar
filtered_final_df = pd.merge(stop_times_with_stops_trips_and_routes_df, calendar_dates_df, on='service_id')

# Filter -> bara fyrir daginn 20240411
filtered_final_current_df = filtered_final_df[filtered_final_df['date'] == 20240411] # finna betri leið service id er unique og nýjasta

# Filter -> óþarfa gögn
df_re_filtered = filtered_final_current_df.drop(['stop_headsign', 'trip_short_name', 'exception_type', 'service_id', 'route_id', 'shape_id', 'date', 'day_pattern', 'pickup_type','location_type'], axis=1)

# raða til að geta sett inn rétta liði
df = df_re_filtered.sort_values(['trip_id', 'stop_sequence'])

stop_id_to_name = stops_df.set_index('stop_id')['stop_name'].to_dict()

# Assign 'next_stop_id' and 'next_arrival_time'
df['next_stop_id'] = df.groupby('trip_id')['stop_id'].shift(-1)
df['next_arrival_time'] = df.groupby('trip_id')['arrival_time'].shift(-1)

# Check if the column has been created and has NaN values that need filling
if 'next_arrival_time' in df.columns:
    df['next_arrival_time'].fillna('End of trip', inplace=True)
else:
    print("Error: 'next_arrival_time' column not found")

# Convert 'next_stop_id' to 'next_stop_name' using the mapping
df['next_stop_name'] = df['next_stop_id'].map(stop_id_to_name)

# Fill missing 'next_stop_id'
df['next_stop_id'].fillna('No next stop', inplace=True)

# Fill missing 'next_stop_name'
df['next_stop_name'].fillna('End of trip', inplace=True)

df.sort_values(['trip_id', 'stop_sequence'], inplace=True)
# setja flögg til að geta brugðist betur við.
df['is_first_stop'] = df.groupby('trip_id').cumcount() == 0
df['is_last_stop'] = df.groupby('trip_id').cumcount(ascending=False) == 0


# Display or further manipulate 'df'
display(df)

/var/folders/fp/txk1x6611xsg8v92_wtymcph0000gn/T/ipykernel_3869/4047662170.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['next_arrival_time'].fillna('End of trip', inplace=True)
/var/folders/fp/txk1x6611xsg8v92_wtymcph0000gn/T/ipykernel_3869/4047662170.py:65: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon,trip_headsign,direction_id,block_id,route_short_name,next_stop_id,next_arrival_time,next_stop_name,is_first_stop,is_last_stop
60,524766,13:27:00,13:27:00,90020295,1,Hlemmur B,64.143719,-21.915081,Hfj. Skarðshlíð,0,4134_1-G,1,90000052.0,13:28:00,Barónsstígur,True,False
153,524766,13:28:00,13:28:00,90000052,2,Barónsstígur,64.144475,-21.918978,Hfj. Skarðshlíð,0,4134_1-G,1,90000821.0,13:29:00,Bíó Paradís,False,False
246,524766,13:29:00,13:29:00,90000821,3,Bíó Paradís,64.145639,-21.924854,Hfj. Skarðshlíð,0,4134_1-G,1,90000054.0,13:30:00,Þjóðleikhúsið,False,False
339,524766,13:30:00,13:30:00,90000054,4,Þjóðleikhúsið,64.146842,-21.930941,Hfj. Skarðshlíð,0,4134_1-G,1,90000055.0,13:32:00,Lækjartorg A,False,False
432,524766,13:32:00,13:32:00,90000055,5,Lækjartorg A,64.147465,-21.936231,Hfj. Skarðshlíð,0,4134_1-G,1,90000056.0,13:33:00,Ráðhúsið,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5574294,529211,15:01:00,15:01:00,90000061,31,Mýrargata,64.151683,-21.949006,Grandi,1,4409_14-F,14,90000062.0,15:03:00,Grandagarður,False,False
5574387,529211,15:03:00,15:03:00,90000062,32,Grandagarður,64.153109,-21.949784,Grandi,1,4409_14-F,14,90000063.0,15:04:00,Grunnslóð,False,False
5574480,529211,15:04:00,15:04:00,90000063,33,Grunnslóð,64.154464,-21.947669,Grandi,1,4409_14-F,14,90000064.0,15:05:00,Fiskislóð,False,False
5574573,529211,15:05:00,15:05:00,90000064,34,Fiskislóð,64.155849,-21.945205,Grandi,1,4409_14-F,14,90000020.0,15:06:00,Grandi,False,False


Búið til net sem samsvarar leiðarkerfinu á virkum degi. Sýnið fjölda hnúta og leggja í
netinu og finnið þá stoppistöð sem hefur hæstu útgráðuna (hvaða stöð og hver er útgráðan).
Netið sem þið búið til má vera hvernig sem er svo lengi sem það sé "pythonskt"í laginu.
Þ.e. við getum ítrað yfir hnúta í netinu og fyrir hvern hnút getum við ítrað yfir granna þess
hnúts. Útfærslan í Java í Algorithms bókinni uppfyllir þessi skilyrði og eftirfarandi einfaldi
python kóði gerir það líka.

```
G = {1: [2,3], 2: [], 3: [1,2] }
for v in G:
  print(u,':')
    for u in G[u,v]:
      print(u,' -> ',v)
```
Ef við viljum bæta við einhverjum upplýsingum þá er hægt að gera það í uppflettitöflu.
Ekki er leyfilegt að nota pakka fyrir net í python á borð við networkx eða sambærilegt.

#### Búum til hakkatöflu fyrir stop_id svo við getum auðveldlega náð í nafn á stoppustöð

In [25]:
# búum til hakkatöflu fyrir stop_id svo við getum auðveldlega náð í nafn á stoppustöð
all_stop_ids = df['stop_id'].unique()
stop_id_to_name = pd.Series(df.stop_name.values, index=df.stop_id).to_dict()

print(stop_id_to_name[90000397])

Árvað


#### Búum til netið G

In [26]:
#### hjálparföll fyrir netagerð
def convert_time_str_to_timedelta(time_str):
    parts = time_str.split(':')
    hours, minutes = int(parts[0]), int(parts[1])
    seconds = int(parts[2]) if len(parts) == 2 else 0
    days, hours = divmod(hours, 24)
    return timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)

def format_hours_to_time(hours):
    if hours is None:
        return None
    total_minutes = int(hours * 60)
    h, m = divmod(total_minutes, 60)
    return f"{h:02d}:{m:02d}"

def create_stop_name_to_ids_map(df):
    stop_name_to_ids = {}
    for index, row in df.iterrows():
        if row['stop_name'] not in stop_name_to_ids:
            stop_name_to_ids[row['stop_name']] = []
        stop_name_to_ids[row['stop_name']].append(row['stop_id'])
    return stop_name_to_ids

stop_name_to_ids = create_stop_name_to_ids_map(df)

In [27]:
G = {}
for _, row in df.iterrows():
    stop_id = row['stop_id']
    stop_name = row['stop_name']
    arrival_time = convert_time_str_to_timedelta(row['arrival_time'])
    departure_time = convert_time_str_to_timedelta(row['departure_time'])
    
    if not row['is_last_stop']:
        next_stop_id = row['next_stop_id']
        next_stop_name = row['next_stop_name']
        next_arrival_time = convert_time_str_to_timedelta(row['next_arrival_time'])
        # travel_time = next_arrival_time - arrival_time
        route_short_name = row['route_short_name']
        trip_headsign = row['trip_headsign']
        stop_lat = row['stop_lat']
        stop_lon = row['stop_lon']

        if stop_id not in G:
            G[stop_id] = []
        G[stop_id].append({
            'stop_name': stop_name,
            'route_short_name': route_short_name,
            'trip_headsign': trip_headsign,
            'arrival_time': arrival_time,
            'departure_time': departure_time,
            'next_stop_id': next_stop_id,
            'next_stop_name': next_stop_name,
            'next_arrival_time': next_arrival_time,
            'stop_lat': stop_lat,
            'stop_lon': stop_lon,
        })

In [28]:
# til að prenta netið
first_stop_name = next(iter(stop_name_to_ids))  # This retrieves the first stop name in the dictionary
print(f"First Stop Name: {first_stop_name}")

stop_ids = stop_name_to_ids[first_stop_name]
# print(f"Stop IDs for '{first_stop_name}': {stop_ids}")

# count = 0
# max_display = 1  # Display only the first 10 stops for brevity
# for stop_id in stop_ids:
#     if stop_id in G and count < max_display:
#         print(f"Node: {stop_id}, Stop Name: {first_stop_name}")
#         for edge in G[stop_id]:
#             print(f"  Edge to {edge['next_stop_name']} with id: {edge['next_stop_id']}")
#             print(f"    On its way to {edge['trip_headsign']}")
#             print(f"    Route: {edge['route_short_name']}")
#             print(f"    Arrival Time: {edge['arrival_time']}")
#             print(f"    Departure Time: {edge['departure_time']}")
#             print(f"    Arrival at next stop: {edge['next_arrival_time']}")
#             print(f"    Latitude: {edge['stop_lat']}, Longitude: {edge['stop_lon']}")
#         count += 1
#     elif count >= max_display:
#         print("Display limit reached.")
#         break

First Stop Name: Hlemmur B


In [45]:
#### spurning hvort megi gera ráð fyrir báðum stoppum. fram og tilbaka. 
#### ég heyrði að lokatalan ætti að vera í kringum 1000 sem myndi meika sens. 2*538 ca 1000


#reikna útgráðu        
out_degrees = {stop_id: len(neighbors) for stop_id, neighbors in G.items()}

# Finna stoppistöð með hæstu útgráðu
max_out_degree_stop = max(out_degrees, key=out_degrees.get)
max_out_degree = out_degrees[max_out_degree_stop]
max_out_degree_stop_name = filtered_final_current_df[filtered_final_current_df['stop_id'] == max_out_degree_stop].iloc[0]['stop_name']

print(f"Stoppistöð með hæstu útgráðu er {max_out_degree_stop_name} með útgráðu {max_out_degree}.")

Stoppistöð með hæstu útgráðu er Mjódd með útgráðu 538.


# 2.3.2 Leit (⋆⋆)
Útfærið leitarreikniritið á netinu eins og lýst var að ofan. Sýnið niðurstöðu leitarinnar þegar
við viljum komast frá Meistaravöllum (ath. það eru tvær stöðvar með þessu nafni) til FB,
kl 11:30.

In [103]:
def convert_time_str_to_timedelta(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return timedelta(hours=hours, minutes=minutes)

def format_hours_to_time(timedelta_obj):
    total_seconds = int(timedelta_obj.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, _ = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}"

def find_route(G, origin_id, destination_id, start_time):
    start_time = convert_time_str_to_timedelta(start_time)
   # entry_count = 0
    pq = []
    #visited = {}
    min_time = {stop_id: timedelta.max for stop_id in G}
    min_time[origin_id] = start_time
    previous_stop = {stop_id: None for stop_id in G}

    heapq.heappush(pq, (start_time, origin_id, []))  # Also push the path taken
    #for start in starts:
     #   pq = [(start_time, entry_count, {'stop': start, 'route': None, 'departure_time': start_time, 'arrival_time': start_time})]
      #  entry_count += 1
       # visited[start] = start_time


    while pq:
        current_time, current_stop_id, path = heapq.heappop(pq)
        
        if current_stop_id == destination_id:
            return path + [(current_stop_id, current_time)], format_hours_to_time(current_time)
        
        for data in G[current_stop_id]:
            departure_time = data['departure_time']
            arrival_time = data['next_arrival_time']
            next_stop_id = data['next_stop_id']
            next_route = data['route_short_name']
            trip_headsign = data['trip_headsign']

            if current_time <= departure_time and arrival_time < min_time[next_stop_id]:
                min_time[next_stop_id] = arrival_time
                previous_stop[next_stop_id] = current_stop_id
                #new_path = path + [{'stop_id': current_stop_id, 'arrival_time': arrival_time, 'route': next_route}]
                heapq.heappush(pq, (arrival_time, next_stop_id, path + [{'stop_id': current_stop_id, 'arrival_time': current_time, 'route': next_route, 'trip_headsign': trip_headsign}]))
                #heapq.heappush(pq, (arrival_time, next_stop_id, new_path))

            if path and path[-1]['route'] != next_route and current_time + timedelta(minutes=2) > departure_time:
                continue
    
    return None, "No route found"

# Function usage example
meistaravellir = find_unique_stop_ids(df, 'Meistaravellir')[0]
fb = find_unique_stop_ids(df, 'FB')[0]
route, arrival_time = find_route(G, meistaravellir, fb, '11:30')
if route:
    '''previous_route = None
    previous_trip_headsign = None
    for i, leg in enumerate(route):
        if isinstance(leg, dict) and 'stop_id' in leg and 'arrival_time' in leg and 'route' in leg:
            stop_id = leg['stop_id']
            arrival_time = leg['arrival_time']
            current_route = leg['route'] 
            stop_name = stop_id_to_name[stop_id]
            formatted_arrival_time = format_hours_to_time(arrival_time)
            trip_headsign =leg.get('trip_headsign', previous_trip_headsign)
         
            print(f"Strætó númer:{current_route} kemur á {stop_name} klukkan {formatted_arrival_time}")
            print(leg)

            # Print change message only if the route has actually changed
            if previous_route is not None and (current_route != previous_route or trip_headsign != previous_trip_headsign):
                print(f"Skiptu yfir í strætó: {current_route} á leið til {trip_headsign}")

            # Update previous_route if it's different
            previous_route = current_route
            previous_trip_headsign = trip_headsign'''

    if isinstance(route[-1], dict):
        final_arrival_time = format_hours_to_time(route[-1]['arrival_time'])
    else:
        _, final_arrival_time = route[-1]
        final_arrival_time = format_hours_to_time(final_arrival_time)

    print(f"Komutími: {final_arrival_time}")


Komutími: 12:36



## 2.3.3 Framsetning (⋆)
Takið löglega leið og búið til leiðarlýsingu, þ.e. hvaða vagn á að taka klukkan hvað og hvar
á að skipta um stöð, svipað og gert er á myndinni að ofan.

In [101]:
meistaravellir = find_unique_stop_ids(df, 'Meistaravellir')[0]
fb = find_unique_stop_ids(df, 'FB')[0]
route, arrival_time = find_route(G, meistaravellir, fb, '11:30')
if route:
    previous_route = None
    previous_stop_name = None
    previous_trip_headsign = None
    stop_count = 0
    start_stop_name = None

    for i, leg in enumerate(route):
        if isinstance(leg, dict) and 'stop_id' in leg and 'arrival_time' in leg and 'route' in leg:
            stop_id = leg['stop_id']
            arrival_time = leg['arrival_time']
            departure_time = leg.get('departure_time', arrival_time)
            current_route = leg['route'] 
            stop_name = stop_id_to_name[stop_id]
            trip_headsign = leg.get('trip_headsign', previous_trip_headsign)
            formatted_arrival_time = format_hours_to_time(arrival_time)
            formatted_departure_time = format_hours_to_time(departure_time)
        
            if previous_route is None:
                start_stop_name = stop_name  # Set the initial stop name
                previous_route = current_route
                previous_stop_name = stop_name
                previous_trip_headsign = trip_headsign
                stop_count = 1
            
                print(f"Byrjar á {stop_name} klukkan {formatted_departure_time}, á leið {current_route} til {trip_headsign}")

            elif current_route == previous_route:
                stop_count += 1

            else:
                if departure_time:
                    print(f"Frá {start_stop_name}, farðu {stop_count} stopp með leið {previous_route}, á leið til {previous_trip_headsign}")
                print(f"Skiptu yfir í strætó {current_route} á stoppistöð: {stop_name} á leið til {trip_headsign} klukkan {formatted_arrival_time}")
                start_stop_name = stop_name
                stop_count = 1  # reset stop count for new route
                previous_trip_headsign = trip_headsign
            
            previous_route = current_route
            previous_stop_name = stop_name

    # Print final arrival details
    if stop_count > 0 and start_stop_name:
        print(f"Frá {start_stop_name}, farðu {stop_count} stopp með leið {previous_route} á leið til {previous_trip_headsign}")
        if isinstance(route[-1], dict):
            final_stop_name = stop_id_to_name[route[+1]['stop_id']]
            final_arrival_time = format_hours_to_time(route[-1]['arrival_time'])
        elif isinstance(route[-1], tuple):
            final_stop_name =stop_id_to_name[route[-1][0]]
            final_arrival_time = format_hours_to_time(route[-1][1])
        else:
            final_stop_name = previous_stop_name
            final_arrival_time = format_hours_to_time(final_arrival_time)

        print(f"Komutími í {final_stop_name} : {final_arrival_time}")
# class Node:  # Hnutur
#     def __init__(self, name: str):
#         self.name = name  # Name of the stop

# class Edge:  # Leggur
#     def __init__(self, start: Node, end: Node, start_time: str, end_time: str):
#         self.start = start  # Starting stop
#         self.end = end  # Ending stop
#         self.start_time = start_time  # Departure time from the start stop
#         self.end_time = end_time  # Arrival time at the end stop
# throstur@arekstur.is 

Byrjar á Meistaravellir klukkan 11:30, á leið 13 til Eiðisgrandi
Frá Meistaravellir, farðu 27 stopp með leið 13, á leið til Eiðisgrandi
Skiptu yfir í strætó 11 á stoppistöð: RÚV á leið til Mjódd klukkan 11:59
Frá RÚV, farðu 8 stopp með leið 11, á leið til Mjódd
Skiptu yfir í strætó 3 á stoppistöð: Mjódd á leið til Sel/Fell klukkan 12:12
Frá Mjódd, farðu 18 stopp með leið 3 á leið til Sel/Fell
Komutími í FB : 12:36


## 2.3.4 Labb (⋆⋆)
Sumar stoppistöðvar eru nálægt hvor annari en ekki hægt að taka strætó á milli þeirra. Breytið netinu og reikniritinu ykkar til að höndla þetta tilfelli, notið GPS hnitin sem eru gefin í gögnunum og veljið hæfilega fjarlægð sem þið eruð til í að láta fólk labba. T.d. er hægt að labba úr 13 í Suðurveri yfir í 4 á Kringlumýrarbraut en 13 stoppar ekki þar. Sýnið dæmi um leið þar sem komutími styttist við að leyfa labb. Þessi aðferð við að tengja saman stöðvar er ekki fullkomin og sýnir að oft er mikilvægara að hafa aðgang að betri gögnum, t.d. eru stoppistöðvar nálægt í planinu en taka langan tíma að labba á milli.

## 2.3.5 Tímamælingar (⋆)
Mælið fjölda fyrirspurna sem aðferðin ykkar getur svarað á sekúndu. Tryggið að mælingarnar blandi stuttum og löngum leiðum með ólíkum tímum.

## 2.3.6 Bestun og gagnagrindur (⋆⋆)
Notið niðurstöður úr tímamælingu til að ná betri afköstum. Hér skiptir máli hvernig framsetningin á netinu er framkvæmd, hvaða gagnagrindur eru notaðar og hvort hægt sé að bæta eitthvað.

## 2.3.7 Færri skiptingar (⋆⋆)
Í einhverjum tilfellum eru margar ólíkar leiðir sem taka jafnlangan tíma, þ.e. seinasti vagninn kemur á áfangastað á sama tíma en fjöldi vagna sem er tekinn er ekki sá sami. Útskýrið hvernig er hægt að breyta reikniritinu til skipta sem sjaldnast um vagn og útfærið þetta á netinu. Sýnið dæmi um strætóferð þar sem niðurstaðan breytist.

## 2.3.8 A(⋆⋆⋆)
Útfærið A∗ reikniritið (sjá kafla 10.8 í https://people.mpi-inf.mpg.de/~mehlhorn/ftp/
NewToolbox/spath.pdf) með því að velja heppilegt fjarlægðarfall f . Sýnið að fjarlægðar-
fallið sem er valið sé löglegt, þ.e. reikniritið skilar alltaf réttum niðurstöðum, og framkvæmið
tímamælingar sem mæla hversu miklu þessi breyting skilar.

## 2.3.9 Pokémon (⋆ ⋆ ⋆)
Finnið þá leið sem tekur mestan fjölda vagna. Athugið að þetta verkefni er mögulega NP-erfitt, takið eftir að netið eins og það er sett upp er ekki DAG (stefnt óhringað net) en það má breyta því í DAG með því að taka út hnútana sem samsvara stoppistöðvum og bæta við hnút fyrir stoppistöð og hvern tímapunkt (x, t), þá setjum við legg frá (x, t) til (x, t′) ef t′ er næsti tímapunktur fyrir þessa stöppistöð og legg sem fara frá (x, t) til v ef vagn fer frá x á tíma t.

Finnið, með einhverju móti, lengstu mögulegu leið sem tekur sem flesta ólíka strætis-
vagnaleiðir.